In [24]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import transforms, datasets

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(False)

In [25]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1,6,kernel_size=5) #1 channel image input and 6 channel image as output using 5x5 kernel
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(6,16,kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.fc1 = nn.Linear(400,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,400)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x),dim=1)
        return x
        

In [26]:
net = LeNet()
print(net)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [27]:
params = list(net.named_parameters())

tot_params = 0
for name, param in params:
    print(name, '--->',param.shape)
    tot_params += np.sum(np.prod(param.size()))
print('\nTot no. of parameters: ',tot_params)

conv1.weight ---> torch.Size([6, 1, 5, 5])
conv1.bias ---> torch.Size([6])
conv2.weight ---> torch.Size([16, 6, 5, 5])
conv2.bias ---> torch.Size([16])
fc1.weight ---> torch.Size([120, 400])
fc1.bias ---> torch.Size([120])
fc2.weight ---> torch.Size([84, 120])
fc2.bias ---> torch.Size([84])
fc3.weight ---> torch.Size([10, 84])
fc3.bias ---> torch.Size([10])

Tot no. of parameters:  61706


In [28]:
apply_transform = transforms.Compose([transforms.Resize(32), transforms.ToTensor()])

Batch = 100
train_set = datasets.MNIST(root='./data', train=True, download=False, transform=apply_transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=Batch, shuffle=True,num_workers=4)
#optimizer = optim.Adam(net.parameters(), lr=0.001)

In [29]:
#for i,data in enumerate(train_loader):
    #inputs,labels = data
    #outputs = net(inputs)

In [30]:
data1 = next(iter(train_set))
input1, label1 = data1

input1 = input1.unsqueeze(0)
pred1 = net(input1)

print(input1.shape)
print(pred1)

torch.Size([1, 1, 32, 32])
tensor([[0.1021, 0.0978, 0.0938, 0.0936, 0.1003, 0.0924, 0.1024, 0.1083, 0.0987, 0.1106]])


In [31]:
print('Input label is: ',label1)
print('Predicted label is: ',pred1.argmax().item())

Input label is:  5
Predicted label is:  9


In [32]:
data = next(iter(train_loader)) #As a batch from the train_loader
inputs,labels = data
preds = net(inputs)

In [33]:
inputs.shape

torch.Size([100, 1, 32, 32])

In [34]:
def get_num_corects(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [35]:
get_num_corects(preds,labels) #number of correct labels were predicted by the Feed Forward method for a batch

11